[View in Colaboratory](https://colab.research.google.com/github/ichara/GTSRB_CapsNet/blob/master/GTSRB_CapsNet.ipynb)

Data: [Keras Tutorial - Traffic Sign Recognition](https://chsasank.github.io/keras-tutorial.html),  
Model: [keras/examples/cifar10_cnn_capsule.py](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn_capsule.py) + Conv2Ds, Denses, etc.

For TensorFlow backend


In [0]:
import numpy as np
from skimage import color, exposure, transform

NUM_CLASSES = 43
IMG_SIZE = 48

def preprocess_img(img):
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # central square(* 0.9) crop
    min_side = min(img.shape[:-1]) * 9 // 10 # Added
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[centre[0] - min_side // 2:centre[0] + min_side // 2,
              centre[1] - min_side // 2:centre[1] + min_side // 2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    return img


In [2]:
from skimage import io
import os
import glob

def get_class(img_path):
    return int(img_path.split('/')[-2])

# root_dir = 'GTSRB/Final_Training/Images/'
root_dir = 'GTSRB/Final_Training/Images/'
imgs = []
labels = []

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img_path = img_path.replace('\\', '/') # Added for Windows OS
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)

X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
import pandas as pd
test = pd.read_csv('GTSRB/GT-final_test.csv', sep=';')

# Load test dataset
X_test = []
y_test = []
i = 0
for file_name, class_id in zip(list(test['Filename']), list(test['ClassId'])):
    img_path = os.path.join('GTSRB/Final_Test/Images/', file_name)
    X_test.append(preprocess_img(io.imread(img_path)))
    y_test.append(class_id)

X_test = np.array(X_test)
y_test = np.array(y_test)


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
from __future__ import print_function
from keras import backend as K
from keras.engine.topology import Layer
from keras import activations
from keras import utils
# from keras.datasets import cifar10
from keras.models import Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

# the squashing function.
# we use 0.5 in stead of 1 in hinton's paper.
# if 1, the norm of vector will be zoomed out.
# if 0.5, the norm will be zoomed in while original norm is less than 0.5
# and be zoomed out while original norm is greater than 0.5.
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


class Capsule(Layer):
    """A Capsule Implement with Pure Keras
    There are two vesions of Capsule.
    One is like dense layer (for the fixed-shape input),
    and the other is like timedistributed dense (for various length input).

    The input shape of Capsule must be (batch_size,
                                        input_num_capsule,
                                        input_dim_capsule
                                       )
    and the output shape is (batch_size,
                             num_capsule,
                             dim_capsule
                            )

    Capsule Implement is from https://github.com/bojone/Capsule/
    Capsule Paper: https://arxiv.org/abs/1710.09829
    """

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            if K.backend() == 'theano':
                o = K.sum(o, axis=1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


Using TensorFlow backend.


In [0]:
tflog_num = 0

In [0]:
# for TensorBoard
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

K.clear_session()

session = tf.Session('')
KTF.set_session(session)

tflog_num += 1
tb_cb = keras.callbacks.TensorBoard(log_dir='tflog' + str(tflog_num), histogram_freq=1)
cbks = [tb_cb]


In [7]:
batch_size = 64
num_classes = NUM_CLASSES
num_dims = 16

K.set_image_dim_ordering('tf')

# A common Conv2D model
input_image = Input(shape=(None, None, 3))
x = Conv2D(64, (3, 3), activation='relu')(input_image)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = Conv2D(256, (3, 3), activation='relu')(x)

"""now we reshape it as (batch_size, input_num_capsule, input_dim_capsule)
then connect a Capsule layer.

the output of final model is the lengths of 10 Capsule, whose dim=16.

the length of Capsule is the proba,
so the problem becomes a 10 two-classification problem.
"""

x = Reshape((-1, 256))(x)
capsule = Capsule(num_classes, num_dims, 3, True)(x)
output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(capsule)
output = Dense(256)(output)
output = Dropout(0.5)(output)
output = Dense(43)(output)
model = Model(inputs=input_image, outputs=output)

# we use a margin loss
adam = keras.optimizers.Adam(lr=0.0001); # Added: 0.001 -> 0.0001
model.compile(loss=margin_loss, optimizer=adam, metrics=['accuracy'])
model.summary()


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, None, 64)    256       
_________________________________________________________________
dropout_1 (Dropout)          (None, None, None, 64)    0         
___________________________________________________________

In [8]:
from sklearn.cross_validation import train_test_split

epochs = 15

X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42)

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by dataset std
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in 0 to 180 degrees
    width_shift_range=0.1,  # randomly shift images horizontally
    height_shift_range=0.1,  # randomly shift images vertically
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    zoom_range=[0.9, 1.1],
    shear_range=0.1
    )

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(X_train, Y_train, batch_size=batch_size),
    epochs=epochs,
    # callbacks=cbks, # for TensorBoard
    validation_data=(X_val, Y_val),
    workers=4)


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Epoch 1/15
491/491 [==============================] - 52s 105ms/step - loss: 0.5587 - acc: 0.2826 - val_loss: 0.2931 - val_acc: 0.6517
Epoch 2/15
234/491 [=============>................] - ETA: 24s - loss: 0.2940 - acc: 0.6614

491/491 [==============================] - 49s 99ms/step - loss: 0.2322 - acc: 0.7475 - val_loss: 0.0728 - val_acc: 0.9308
Epoch 3/15
362/491 [=====================>........] - ETA: 11s - loss: 0.0952 - acc: 0.9246

491/491 [==============================] - 48s 98ms/step - loss: 0.0879 - acc: 0.9322 - val_loss: 0.0238 - val_acc: 0.9807
Epoch 4/15
408/491 [=======================>......] - ETA: 7s - loss: 0.0492 - acc: 0.9692

491/491 [==============================] - 49s 99ms/step - loss: 0.0475 - acc: 0.9703 - val_loss: 0.0113 - val_acc: 0.9902
Epoch 5/15
428/491 [=========================>....] - ETA: 5s - loss: 0.0299 - acc: 0.9831

...

491/491 [==============================] - 49s 100ms/step - loss: 0.0084 - acc: 0.9952 - val_loss: 0.0036 - val_acc: 0.9971
Epoch 12/15
436/491 [=========================>....] - ETA: 5s - loss: 0.0066 - acc: 0.9963

491/491 [==============================] - 48s 98ms/step - loss: 0.0065 - acc: 0.9964 - val_loss: 0.0041 - val_acc: 0.9972
Epoch 13/15
434/491 [=========================>....] - ETA: 5s - loss: 0.0073 - acc: 0.9955

491/491 [==============================] - 48s 98ms/step - loss: 0.0072 - acc: 0.9956 - val_loss: 0.0024 - val_acc: 0.9978
Epoch 14/15
425/491 [========================>.....] - ETA: 6s - loss: 0.0049 - acc: 0.9974

491/491 [==============================] - 49s 100ms/step - loss: 0.0050 - acc: 0.9972 - val_loss: 0.0071 - val_acc: 0.9926
Epoch 15/15
429/491 [=========================>....]

491/491 [==============================] - 49s 100ms/step - loss: 0.0063 - acc: 0.9962 - val_loss: 0.0029 - val_acc: 0.9976


In [9]:
# predict and evaluate
y_proba = model.predict(X_test)
y_pred = y_proba.argmax(axis=-1)
acc = np.sum(y_pred == y_test) / np.size(y_pred)
print("Test accuracy = {}".format(acc))


Test accuracy = 0.9832937450514647


In [0]:
# format for TSR-Analysis: 00000.ppm;16
np.savetxt('y_pred_' + str(tflog_num) + '.csv', 
           np.column_stack((np.arange(len(y_pred)), y_pred)), fmt='%05d.ppm;%.0f')
